# Gait detection
This notebook shows how to predict gait probability using extracted features from windowed accelerometry data as input.

## Modules

In [1]:
# Automatically reload modules
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import pandas as pd
import tsdf

from dbpd import DataColumns
from dbpd.gait_analysis_config import GaitDetectionConfig

## Constants

In [12]:
# Cell has the tag 'parameters'
path_to_data = '../../../tests/data'
input_path = os.path.join(path_to_data, '3.extracted_features', 'gait')
output_path = os.path.join(path_to_data, '4.predictions', 'gait')


In [13]:
path_to_classifier_input = os.path.join(path_to_data, '0.classifiers', 'gait')

config = GaitDetectionConfig()

## Load data

#### Features

In [14]:
classifier_path = os.path.join(input_path, config.meta_filename)
print(classifier_path)
metadata_dict = tsdf.load_metadata_from_path(classifier_path)
metadata_time = metadata_dict[config.time_filename]
metadata_samples = metadata_dict[config.values_filename]

df = tsdf.load_dataframe_from_binaries([metadata_time, metadata_samples], tsdf.constants.ConcatenationType.columns)

df.head(2)

../../../tests/data/3.extracted_features/gait/gait_meta.json


,time,grav_accelerometer_x_mean,grav_accelerometer_y_mean,grav_accelerometer_z_mean,grav_accelerometer_x_std,grav_accelerometer_y_std,grav_accelerometer_z_std,accelerometer_x_power_below_gait,accelerometer_y_power_below_gait,accelerometer_z_power_below_gait,...,cc_7_acc,cc_8_acc,cc_9_acc,cc_10_acc,cc_11_acc,cc_12_acc,cc_13_acc,cc_14_acc,cc_15_acc,cc_16_acc
0,0.0,-0.425016,0.374486,0.133731,0.224174,0.217684,0.395868,-2.686308,-1.575942,-1.272302,...,5.901427,7.562094,3.293089,2.331375,2.694117,1.800090,1.468428,3.353820,3.091814,2.214879
1,1.0,-0.488237,0.466395,0.228077,0.139916,0.162138,0.419461,-2.752232,-1.538532,-1.352517,...,8.497128,5.700717,1.320182,0.128242,2.470097,3.167589,3.097970,3.203964,1.507816,2.076287


#### Classifier

In [5]:
clf = pd.read_pickle(os.path.join(path_to_classifier_input, config.classifier_file_name))

with open(os.path.join(path_to_classifier_input, config.thresholds_file_name), 'r') as f:
    thresholds_str = f.read()

threshold = np.mean([float(x) for x in thresholds_str.split(' ')])

## Prepare data

In [6]:
clf.feature_names_in_ = [f'{x}_power_below_gait' for x in config.l_accel_cols] + \
                        [f'{x}_power_gait' for x in config.l_accel_cols] + \
                        [f'{x}_power_tremor' for x in config.l_accel_cols] + \
                        [f'{x}_power_above_tremor' for x in config.l_accel_cols] + \
                        ['std_norm_acc'] + [f'cc_{i}_acc' for i in range(1, 17)] + [f'grav_{x}_{y}' for x in config.l_accel_cols for y in ['mean', 'std']] + \
                        [f'{x}_dominant_frequency' for x in config.l_accel_cols]

X = df.loc[:, clf.feature_names_in_]

## Predict gait

In [7]:
df['pred_gait_proba'] = clf.predict_proba(X)[:, 1]
df['pred_gait'] = df['pred_gait_proba'] > threshold

## Store predictions

In [8]:
metadata_samples.__setattr__('file_name', 'gait_values.bin')
metadata_samples.__setattr__('file_dir_path', output_path)
metadata_time.__setattr__('file_name', 'gait_time.bin')
metadata_time.__setattr__('file_dir_path', output_path)

metadata_samples.__setattr__('channels', ['pred_gait_proba'])
metadata_samples.__setattr__('units', ['probability'])
metadata_samples.__setattr__('data_type', np.float32)
metadata_samples.__setattr__('bits', 32)

metadata_time.__setattr__('channels', ['time'])
metadata_time.__setattr__('units', ['relative_time_ms'])
metadata_time.__setattr__('data_type', np.int32)
metadata_time.__setattr__('bits', 32)

In [9]:
if not os.path.exists(output_path):
    os.makedirs(output_path)

# store binaries and metadata
tsdf.write_dataframe_to_binaries(output_path, df, [metadata_time, metadata_samples])
tsdf.write_metadata([metadata_time, metadata_samples], 'gait_meta.json')